<a href="https://colab.research.google.com/github/deekshakoul/Sentence-classification-via-Attention-mechanism/blob/main/SA_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model leverages the power of the self-attention mechanism and CNN for sentence classification on multiple datasets. 

In [ ]:
!git clone https://github.com/deekshakoul/sent-conv-torch.git

Cloning into 'sent-conv-torch'...
remote: Enumerating objects: 477, done.
remote: Total 477 (delta 0), reused 0 (delta 0), pack-reused 477
Receiving objects: 100% (477/477), 58.85 MiB | 13.17 MiB/s, done.
Resolving deltas: 100% (309/309), done.


In [ ]:
import re,string
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.model_selection import train_test_split

import math
import os
import copy
#=========================pytorch========================#
import torch   
from torchtext import data ,vocab
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
SEED = 134
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print("cuda? ",torch.cuda.is_available())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
cuda?  True


In [ ]:
def clean_str(string):
  string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
  string = re.sub(r"\'s", " \'s", string) 
  string = re.sub(r"\'ve", " \'ve", string) 
  string = re.sub(r"n\'t", " n\'t", string) 
  string = re.sub(r"\'re", " \'re", string) 
  string = re.sub(r"\'d", " \'d", string) 
  string = re.sub(r"\'ll", " \'ll", string) 
  string = re.sub(r",", " , ", string) 
  string = re.sub(r"!", " ! ", string) 
  string = re.sub(r"\(", " ( ", string) 
  string = re.sub(r"\)", " ) ", string) 
  string = re.sub(r"\?", " ? ", string) 
  string = re.sub(r"\s{2,}", " ", string)    
  return string.strip().lower()

In [ ]:
from pre_process import create_alldata_csv
df = create_alldata_csv('subj.all') 
#subj.all , custrev.all , rt-polarity.all , mpqa.all
df=df.sample(frac=1)
print(df)

In [ ]:
######################Dont run again and again###########################
!cp /content/drive/My\ Drive/iisc/summer/datasets/glove.840B.300d.zip /content
!unzip /content/glove.840B.300d.zip
import os
emb_path = '/content'
vec = vocab.Vectors(os.path.join(emb_path, 'glove.840B.300d.txt'), cache=emb_path,unk_init = torch.Tensor.normal_)

Archive:  /content/glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


100%|█████████▉| 2195956/2196017 [05:10<00:00, 7613.32it/s]

In [ ]:
'''
  Take data from fulldata.csv
  Build_vocab: give indexing to words(tokens)
  <UNK>:0 and <PAD>:1
  Using vec(glove), create embeddings given the token 
'''
TEXT = data.Field(lower=True,batch_first=True,include_lengths=True)#,stop_words=stop)tokenize='spacy',
LABEL = data.LabelField(dtype = torch.int,batch_first=True)
fields = [('text',TEXT),('label', LABEL)]
all_data = data.TabularDataset(skip_header = True,path='/content/fulldata.csv',format = 'csv', fields=fields)
TEXT.build_vocab(all_data,vectors = vec ) #all_data has tokens(words) and they will be mapped to embeddings of glove..if word is in glove
LABEL.build_vocab(all_data)
word_embeddings = TEXT.vocab.vectors
word_embeddings[1] = torch.zeros(word_embeddings.shape[1])
print("Size of TEXT vocabulary:",len(TEXT.vocab)) #total words
print("===========\n",TEXT.vocab.vectors)
#u will notice <UNK>:0 and <PAD>:1 initialized randomly.....rest words or indices will always remain same

Size of TEXT vocabulary: 21324
 tensor([[ 0.1040,  0.8886,  1.7246,  ...,  0.2860, -0.0557, -1.4377],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0828,  0.6720, -0.1499,  ..., -0.1918, -0.3785, -0.0659],
        ...,
        [ 0.8111,  0.3765,  0.3113,  ...,  0.3401, -0.1715,  0.0132],
        [-0.1490, -0.8729, -0.2281,  ..., -0.0667, -0.3832,  0.3249],
        [ 0.6228, -0.0159,  0.3422,  ...,  0.2939, -0.1054, -0.8087]])


In [ ]:
###FOLDS GENERATION#####
from sklearn.model_selection import StratifiedKFold, KFold
cv = 4
kf = StratifiedKFold(cv, True, SEED)

folds=[]
for tr, tt in kf.split(df["text"], df["label"]):
    #print(train_index, test_index)
    #print(len(train_index), len(test_index))
    folds.append((list(tr), list(tt)))

In [ ]:
def build_criterion(): 
  criterion = nn.CrossEntropyLoss()
  criterion = criterion.to(device)
  return criterion
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#GLOBAL PARAMS#
global testing
testing =  False
SRC_PAD_IDX=TEXT.vocab.stoi[TEXT.pad_token]
d_model = 300
d_v = 128
d_k = 128
n_heads = 4
batch = 16
output_size = 2
EPOCHS= 200
clip=1e-3
# lrate = 1e-4
# wd=3e-5
#GOOD STILL
lrate = 1.5e-5
wd=3e-5
#lrate = 1e-5
#wd=1e-4
# dropouts#
d1=0.5#dropout to main network
d2=0.5
d3=0.5#scaled dot product
d4=0.5#positional-encoding
#CNN PARAMS#
Ci= 1
Co=100
Ks=[3,4,5]
max_kernel_size = 5

best_acc_overall =0

for i, (tr_ix, tt_ix) in enumerate(folds):
  print("Fold ==> ",i+1)
  train_data = [item for i,item in enumerate(all_data) if i in tr_ix]
  test_data = [item for i,item in enumerate(all_data) if i in tt_ix]
  train = data.Dataset(train_data,fields)
  test = data.Dataset(test_data,fields)
  print("train size: ", len(train_data), " and test size: ", len(test_data))
  train_itr,test_itr = data.BucketIterator.splits(
                (train, test), 
                batch_size = batch,
                sort_key = lambda x: len(x.text),
                sort_within_batch=True,
                device = device,
                shuffle=True
              )   
  
  model = MainNetwork()
  model = model.to(device)
  print(f'The model has {count_parameters(model):,} trainable parameters')
  criterion = build_criterion()
  optimizer = torch.optim.Adam(model.parameters(),lr=lrate, weight_decay=wd)
 
  best_acc = 0.0
  best_loss = 100.0
  test_loss = []
  tr_loss = []
  test_acc = []
  tr_acc = []  
 
  for ep in range(EPOCHS):
    train_loss, train_acc = training(model,train_itr,criterion,batch, optimizer,clip=clip)
    eval_loss, eval_acc = evaluation(model,test_itr,batch)
    print(f'Epoch: {ep+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f} %, Test Loss: {eval_loss:3f}, test Acc: {eval_acc:.2f}%')    
 
    test_loss.append(eval_loss)
    tr_loss.append(train_loss)
    test_acc.append(eval_acc)
    tr_acc.append(train_acc)

    if eval_acc > best_acc:
      best_acc = eval_acc
      
    if eval_loss < best_loss:
      best_loss = eval_loss

  if  best_acc_overall < best_acc:  #best_acc is for every fold
    best_acc_overall = best_acc
    model_copy = copy.deepcopy(model)
    print("For fold ",i+1," given ",cv," folds", " best accuracy for this fold is  ",best_acc)


  plot_losses_per_fold(tr_loss,test_loss,i+1)
  plot_losses_per_fold(tr_acc,test_acc,i+1,"ACC")

In [ ]:
best_acc_overall

81.35593220338983

In [ ]:
def plot_losses_per_fold(tr, tt,fold,c="LOSS"):
  epoch_count = range(1, len(tr) + 1)
  name = 'MR' + "_CV" + str(fold)  + "_" + c
  plt.plot(epoch_count, tr, 'r--')
  plt.plot(epoch_count, tt, 'b-')
  if c == "LOSS":
    plt.legend(['Training Loss', 'Test Loss'])
  else:
    plt.legend(['Training Accuracy', 'Test Accuracy'])
  plt.xlabel('Epoch')
  plt.ylabel('Loss') 
  if c=="ACC":
    plt.ylabel('Accuracy') 
  
  plt.xticks(np.arange(min(epoch_count), max(epoch_count)+1, 40.0))
  # plt.ylim(-5, 5)
  plt.savefig(name)
  plt.close()

In [ ]:
#--------------------------------------******TESTING PURPOSES*******------------------------------------#
dd = TEXT.vocab.stoi
k = list(dd.keys())
v = list(dd.values())
def get_sent(text):
  sent=[]  
  for i in text:
    sent.append(k[v.index(i)])
  return sent
def get_indices_sent(text):
  ind=[]
  tokens = text.split()
  for i in range(len(tokens)):  
    if tokens[i] in k:
      ind.append(dd[tokens[i]])
    else:
      ind.append(TEXT.vocab.stoi[TEXT.unk_token])
  return ind      
def make_batch(sent):
  l=[]
  m=[]
  max = 0
  for i in range(len(sent)):
    ind = get_indices_sent(clean_str(sent[i]))
    l.append(ind)
    if len(ind)>max:
      max=len(ind)
    m.append(len(ind))

  for i in range(len(l)):
    length_sent = len(l[i])
    if length_sent != max:
      l[i].extend([TEXT.vocab.stoi[TEXT.pad_token]]*(max - length_sent))
  return torch.LongTensor(l), torch.LongTensor(m)

In [ ]:
clean_str(sentences[0])

"this movie does n't deserve the energy it takes to describe how bad it is"

In [ ]:
vocab_dim=len(dd)
embed_dim = d_model
model = MainNetwork()
model = model.to(device)

In [ ]:
global attn_wts
testing = True

sentences =["great quality picture and features"] #CR

txt, txt_len = make_batch(sentences)
sts = len(sentences)
model.eval()

with torch.no_grad():
  if torch.cuda.is_available():
    txt = txt.cuda()
    txt_len = txt_len.cuda()
  predictions = model(txt)
  print(predictions)
  print(torch.max(predictions, 1)[1].view(sts).data,"\n \n")
  print("================\n",attn_wts) #[batch_size x n_heads x words x words ]

In [ ]:
xx =clean_str(sentences[0]).split()
xx

['the', 'customer', 'support', 'is', 'pathetic']

In [ ]:
sns.heatmap(attn_wts[0][3].cpu(),xticklabels=xx, yticklabels=xx)

In [ ]:
class MainNetwork(nn.Module):
  def __init__(self):
    super(MainNetwork,self).__init__(); 
    self.embedding = nn.Embedding.from_pretrained(word_embeddings,padding_idx= SRC_PAD_IDX,freeze=True) #freeze=True: no learnable params
    self.pos_embedding = PositionalEncoding(d_model)
    self.self_attn = MultiHeadAttention()

    self.layer_norm = nn.LayerNorm(d_model)#, eps=1e-6)
    self.dropout = nn.Dropout(p=d1)

    ##--------CNN LAYER to get final output--------------##
    self.convs = nn.ModuleList([nn.Conv2d(Ci, Co, (K, d_model)) for K in Ks])
    self.fc1 = nn.Linear(len(Ks) * Co, output_size)
    
      
  def forward(self,text):    
    embeds = self.pos_embedding(self.embedding(text)* math.sqrt(d_model)) 
    #embeds = self.dropout(embeds)
    
    text_mask = get_attn_pad_mask(text,text) # [batch, seq_len , seq_len] --> check for some problems
    
    out,attn =  self.self_attn(embeds, embeds, embeds, text_mask)  #[batch, seq_len, d_model] , [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
    out =  self.layer_norm(out + embeds)
   
    ##------------CNN Layer---------------##
    if out.size()[1] < max_kernel_size:
      m = nn.ZeroPad2d((0, 0, 0, max_kernel_size-out.size()[1]))
      out = m(out)
      
    out = out.unsqueeze(1)  # (N, Ci, W, D)
    out  = [F.relu(conv(out)).squeeze(3) for conv in self.convs]  # [(N, Co, W), ...]*len(Ks)
    out = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in out]  # [(N, Co), ...]*len(Ks)
    out = torch.cat(out, 1)
    out = self.dropout(out) 
    logit = self.fc1(out)  # (N, C)

    global testing,attn_wts
    if testing:
      attn_wts = attn

    return logit  

def get_attn_pad_mask(seq_q, seq_k): #SEE EXPLANATION#1
  assert seq_q.dim() == 2 and seq_k.dim() == 2
  b_size, len_q = seq_q.size()
  b_size, len_k = seq_k.size()
  pad_attn_mask = seq_k.data.eq(SRC_PAD_IDX).unsqueeze(1)  # b_size x 1 x len_k
  return pad_attn_mask.expand(b_size, len_q, len_k)  # b_size x len_q x len_k #Every sentence --> seq_len x seq_len --> why bec needed to be same dimension when dot(QK)
 

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self):
    super(MultiHeadAttention, self).__init__()
    self.W_Q = nn.Linear(d_model, d_k*n_heads) #to get query :: each word :embed_dim/d_model --> dk >>for one head
    self.W_K = nn.Linear(d_model, d_k*n_heads)
    self.W_V = nn.Linear(d_model, d_v*n_heads)
    self.linear = nn.Linear(n_heads * d_v, d_model)   #this might not be required
    # self.layer_norm = nn.LayerNorm(d_model)

  def forward(self,Q,K,V,mask):
    #q:[batch, seq_len, embed_dim] , len_q=seq_len=len_k
    residual, batch_size = Q, Q.size(0)
    
    q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
    '''
    self.W_Q(Q) : [batch,seq_len, dk*n_heads]
    .view(): [batch,seq_len, n_heads, dk]
    .transpose(): [batch,n_heads, seq_len,dk]
    STAGE 1 of getting every word as dk --> Q of every word
    '''

    k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k] 
    v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

    #for every head, we have Q,K,V for all words i.e the len_k/len_q are same?
    mask = mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]
    #whatever was masking earlier is just replicated for nheads for every sentence, remember masking for eavery sentence is of shape:words x words

    context, attn_scores = ScaledDotProduct()(q_s, k_s, v_s, mask)
    '''
    we do scaled dot product "nheads" times  -> got Z for every head now,  Z shape is [batch, ]
    context: [batch, nheads, seq_len(len_q/len_k), dv], here dv comes from v_s
    attn_scores: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]

    we have nheads*dv = d_model [in vaswani]  --> (AA)
    '''
    context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) #using (AA)
    #[batch, seq_len, nheads*dv or d_model]

    out = self.linear(context)


    
    return out, attn_scores #[batch, seq_len, output_size]

In [ ]:
class ScaledDotProduct(nn.Module):
  "Implement Attention as in Transformer-Vaswani "
  def __init__(self):
    super(ScaledDotProduct, self).__init__()
    self.dropout = nn.Dropout(d3)
    '''
    query: [batch, nheads, seq_len,dk]
    key: [batch, nheads, seq_len,dk]
    qkT : [batch, nheads, words,words] 
        i.e for every head: we have attention vector for each word 
    '''
  def forward(self, query, key, value, mask=None):
    scores = torch.matmul(query, key.transpose(-2, -1))/np.sqrt(d_k) #its necessary to write -1,-2  
    #[batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
    
    if mask is not None:
      scores.masked_fill_(mask, -1e9)   #mask is same dimension as scores? yes

    attn =  nn.Softmax(dim=-1)(scores) #softmax over last dimension - per each word, find softmax of probs over each word 
    output = torch.matmul(attn, value)
    return output,attn

In [ ]:
#http://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding 
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(d4)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1) #1-D tensor containing 0,1,2,....,max_len
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe) 
        #register a buffer that should not to be considered a model parameter,
        #saved as part of state_dict along with model params but is not learned
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        #print(x.shape)
        return self.dropout(x)

In [ ]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def training(model, iterator, criterion, batch_size,optimizer,clip):
  # print("training ")
  epoch_loss = 0.0
  epoch_accuracy = 0.0
  #optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters())) # some of the parameters you give the optimizer do not require gradients, and so he don’t know how to handle them
  
  model.train()
  for batch in iterator:
    texts,text_lengths = batch.text 
    target = batch.label.type(torch.LongTensor)

    if  texts.shape[0]!=batch_size:
      continue;

    if torch.cuda.is_available():
      texts = texts.cuda()
      target = target.cuda()    

    optimizer.zero_grad() 
    
    predictions_batch  =  model(texts) #[batch, output_size]
 
    loss = criterion(predictions_batch, target)
    num_corrects = (torch.max(predictions_batch, 1)[1].view(target.size()).data == target.data).float().sum()
    acc = 100.0 * num_corrects/len(batch)

    loss.backward()    
    clip_gradient(model, clip)   
    
    optimizer.step()   

    epoch_loss += loss.item()  
    epoch_accuracy += acc.item()     
    
  return epoch_loss / len(iterator), epoch_accuracy / len(iterator)  

In [ ]:
def evaluation(model, iterator, batch_size):
    total_epoch_loss = 0
    total_epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            text, text_lengths = batch.text

            target = batch.label.type(torch.LongTensor)
            if  text.shape[0]!=batch_size:
              continue;
            if torch.cuda.is_available():
              text = text.cuda()
              target = target.cuda()    

            predictions = model(text)
            loss = criterion(predictions, target)
            #acc = binary_accuracy(predictions, batch.label)
            num_corrects = (torch.max(predictions, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()
            
    return total_epoch_loss/len(iterator), total_epoch_acc/len(iterator)


In [ ]:
#Exaplanation of Masking - EXPLANATION#1
#for dot(QK) --> #words x #words for ONE sentence
tt = torch.tensor([
                   [2,4], #sent1 - 3words
                   [6,1]
                  #  [9,1,1]
])
tt.size() #[3,3]
def get_attn_pad_mask(seq_q, seq_k):
  assert seq_q.dim() == 2 and seq_k.dim() == 2
  b_size, len_q = seq_q.size()
  b_size, len_k = seq_k.size()
  pad_attn_mask = seq_k.data.eq(1).unsqueeze(1)  # b_size x 1 x len_k
  return pad_attn_mask.expand(b_size, len_q, len_k)  # b_size x len_q x len_k

mk = get_attn_pad_mask(tt,tt)  
print(mk) #[batch, seq_len , seq_len] i.e  [batch, words, words]
print(mk.size())
'''
why should we calculate word attention wrt to padding ?
Ex in this, sent2 have last token as PAD
then in matrix equivalent to dot(QK)
w1,w2 should not have any attention wrt to pad
BUT op is:
    [[False, False,  True],
     [False, False,  True],
     [False*, False*,  True]],
'''

tensor([[[False, False],
         [False, False]],

        [[False,  True],
         [False,  True]]])
torch.Size([2, 2, 2])


'\nwhy should we calculate word attention wrt to padding ?\nEx in this, sent2 have last token as PAD\nthen in matrix equivalent to dot(QK)\nw1,w2 should not have any attention wrt to pad\nBUT op is:\n    [[False, False,  True],\n     [False, False,  True],\n     [False*, False*,  True]],\n'